In [1]:
import sys
sys.path.append("..")
from src.Eval import compute_results_jsonl, compute_results_txt
from src.Utils import get_true_pred_entities, read_IOB_file
from baseline.parse_output import get_results_overall_table, get_results_table, parse_preds_baseline

BASELINES = ["bert-large-uncased", "roberta-large", "mpnet-base"]
OUTPUT_PATH_REDDIT = "../baseline/music-ner-eacl2023/output/dataset"


# Overall
## Baseline: BERT etc.

In [2]:
for i in range(1,5):
    print(get_results_overall_table(BASELINES, OUTPUT_PATH_REDDIT + str(i)).loc["macro", "strict", "f1"])


Model
bert-large-uncased    0.780469
mpnet-base            0.768840
roberta-large         0.753986
Name: (macro, strict, f1), dtype: float64
Model
bert-large-uncased    0.806217
mpnet-base            0.829565
roberta-large         0.782475
Name: (macro, strict, f1), dtype: float64
Model
bert-large-uncased    0.772230
mpnet-base            0.770317
roberta-large         0.759375
Name: (macro, strict, f1), dtype: float64
Model
bert-large-uncased    0.692649
mpnet-base            0.704873
roberta-large         0.715011
Name: (macro, strict, f1), dtype: float64


## LLaMa-3

In [3]:
for i in range(1,5):
    try:
        print(f"\nDataset{i}: ")
        data_llm_zeroshot = compute_results_jsonl(
            f"../baseline/music-ner-eacl2023/data/dataset{i}/test.bio", 
            f"../output/reddit/llama3_dataset{i}_simple2_4shot.jsonl")
    except:
        print(f"Dataset{i} Does not work")
        


Dataset1: 
Dataset1 Does not work

Dataset2: 


100%|██████████| 600/600 [00:00<00:00, 8917.22it/s]


Retag IOBs...


100%|██████████| 600/600 [00:00<00:00, 22795.54it/s]

INFO: 1 non matching IOB taglists.


...

INFO: 3 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.68623   0.586873  0.632674        304           40          0       174          99         518       443
ent_type     0.75395   0.644788  0.695109        334           10          0       174          99         518       443
exact        0.708804  0.606178  0.653486        314           30          0       174          99         518       443

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.73251   0.607509  0.664179        178           10          0       105          55 

100%|██████████| 751/751 [00:00<00:00, 9621.79it/s]


Retag IOBs...


100%|██████████| 751/751 [00:00<00:00, 24410.62it/s]
2024-06-24 13:03:54 root INFO: Imported 750 predictions for 750 true examples


INFO: 1 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.690299  0.568356  0.62342         370           62          0       219         104         651       536
ent_type     0.755597  0.62212   0.682393        405           27          0       219         104         651       536
exact        0.740672  0.609831  0.668913        397           35          0       219         104         651       536

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.743243  0.575916  0.648968        220           24          0       138          52 

In [4]:
a,b = read_IOB_file("../baseline/music-ner-eacl2023/data/dataset1/test.bio")


# WHY IS MORE PREDICTED?


### Takeaways from Overall
- at least for few-shot, BERT-like models seem to be better than LLaMa-3

# Error Analysis

In [5]:
data = get_true_pred_entities(
    "bert-large-uncased", "../baseline/music-ner-eacl2023/output/dataset" + str(4)).dropna(
        how="all")

mask_artist = (data.Artist_true != data.Artist_pred) & ~(data.Artist_true.isna() & data.Artist_pred.isna())
mask_woa = (data.WoA_true != data.WoA_pred) & ~(data.WoA_true.isna() & data.WoA_pred.isna())


## Errors at WoA

In [6]:
data.loc[mask_woa, ["text", "WoA_true", "WoA_pred"]]


,text,WoA_true,WoA_pred
6,songs similar to melody noir patrick watson,[melody noir],[melody]
18,a similar emotional production vibe to molly f...,[molly],[lil dicky]
19,need music like the times they are a changin,[the times they are a changin],[they are a changin]
44,asesina kidd sowlo,[asesina],[sowlo]
54,peach pit black licorice,[black licorice],NaN
...,...,...,...
716,recommendations for music like this poolside r...,[harvest moon],"[poolside, harvest moon]"
719,hip hop similar to runnin from the creed 2 album,"[runnin, the creed 2]","[runnin, creed 2]"
720,looking for the genre of searching youth 83,[searching],NaN
730,songs like vincent or imagine,"[vincent, imagine]",NaN


## Errors at Artist

In [8]:
data.loc[mask_artist, ["text", "Artist_true", "Artist_pred"]]


,text,Artist_true,Artist_pred
6,songs similar to melody noir patrick watson,[patrick watson],"[noir, patrick watson]"
11,request | super heavy dubstep i believe that i...,NaN,[deathstep]
18,a similar emotional production vibe to molly f...,"[brandon urie, lil dicky]","[molly, brandon urie]"
19,need music like the times they are a changin,NaN,[the times]
31,grundge elliot smith type of band,[elliot smith],"[grundge, elliot smith]"
...,...,...,...
716,recommendations for music like this poolside r...,"[poolside, neil young]",[neil young]
720,looking for the genre of searching youth 83,[youth 83],[searching youth]
730,songs like vincent or imagine,NaN,"[vincent, imagine]"
743,gently wind | joshua speedy,[joshua speedy],NaN


# Ideas
### Stabilize
- Always: "Does the entity X refer to a Work of Art or a musical Artist?"
- If no entity found: "Is there no entity referring to a Work of Art (WoA) or a musical Artist?"
### Understandability
- If there is a WoA in this text, how did you identify it?
- If there is an Artist in this text, how did you identify it?
- Did you use linguistic knowledge to identify entities or did you 


In [9]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.Wrapper import LlamaWrapper

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama = LlamaWrapper(model_id=model_id)


/data/miniconda3/envs/torch21/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/miniconda3/envs/torch21/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-06-24 13:25:07 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2024-06-24 13:25:07 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2024-06-24 13:25:08 accelerate.utils.modeling INFO: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loa

In [29]:
system_prompt = "You are an excellent linguist. "
utt = "melody"
sentence = "songs similar to melody noir song by patrick watson"
task = f"The utterance '{utt}' in the sentence '{sentence}' refers to (a) 'WoA' (eg. song title, album name) or (b) an 'Artist'?. "
user_prompt = system_prompt + task
print(user_prompt)
schema =    {
            "type": "object",
            "properties": {
                "answer": {"type": "string"},
                "full_utterance": {"type": "string"},
                "explanation": {"type": "string"}
                }
            }

llama.prompt_to_json(
    #system_prompt=system_prompt,
    prompt=system_prompt + user_prompt,
    json_schema=schema,
    temperature=0.00001
)


You are an excellent linguist. The utterance 'melody' in the sentence 'songs similar to melody noir song by patrick watson' refers to (a) 'WoA' (eg. song title, album name) or (b) an 'Artist'?. 


{'answer': 'b',
 'full_utterance': 'songs similar to melody noir song by patrick wat',
 'explanation': "The word'melody' in the sentence refers"}